In [1]:
  !pip install wandb


In [2]:
import wandb


In [3]:
!wandb login 58a0b576fd5221cd0d63b154deaabbe535e853c6

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [9]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'embedding_dim': {'values': [32,64,128,256]},
        'hidden_dim': {'values': [32,64, 128,256 ]},
        'enc_layers': {'values': [1, 2,3]},
        'dec_layers': {'values': [1, 2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.2, 0.3,0.5]},
        'epochs': {'values': [10, 15]},
        'beam_size': {'values': [1, 3, 5]}
    }
}
# Sweep
# =======================
default_config = {
    'embedding_dim': 32,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1
}


# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder and Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

# =======================
# Seq2Seq Model with Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Train Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Val Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="dakshina-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
    dev_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
    train_data = TransliterationDataset(train_path, inp_vocab, out_vocab)
    dev_data = TransliterationDataset(dev_path, inp_vocab, out_vocab)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
    decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch+1})

# =======================

if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="dakshina-transliteration")
    wandb.agent(sweep_id, function=main,count=30)


Create sweep with ID: v20mtq8o
Sweep URL: https://wandb.ai/manglesh_dl_ass3/dakshina-transliteration/sweeps/v20mtq8o


wandb: Agent Starting Run: wsb4nmzt with config:
wandb: 	beam_size: 3
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 3
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 32
wandb: 	enc_layers: 1
wandb: 	epochs: 15
wandb: 	hidden_dim: 256


Epoch 1/15
Train Loss: 2.0825, Acc: 39.08%
Val Loss: 1.3258, Acc: 58.51%
Epoch 2/15
Train Loss: 0.9195, Acc: 71.55%
Val Loss: 0.9491, Acc: 69.49%
Epoch 3/15
Train Loss: 0.6898, Acc: 78.14%
Val Loss: 0.8675, Acc: 71.97%
Epoch 4/15
Train Loss: 0.5977, Acc: 80.81%
Val Loss: 0.8566, Acc: 72.93%
Epoch 5/15
Train Loss: 0.5505, Acc: 82.18%
Val Loss: 0.8085, Acc: 73.42%
Epoch 6/15
Train Loss: 0.5183, Acc: 82.97%
Val Loss: 0.8133, Acc: 74.15%
Epoch 7/15
Train Loss: 0.4930, Acc: 83.72%
Val Loss: 0.8226, Acc: 74.18%
Epoch 8/15
Train Loss: 0.4748, Acc: 84.11%
Val Loss: 0.8079, Acc: 75.05%
Epoch 9/15
Train Loss: 0.4575, Acc: 84.62%
Val Loss: 0.8134, Acc: 74.60%
Epoch 10/15
Train Loss: 0.4436, Acc: 84.93%
Val Loss: 0.8148, Acc: 74.71%
Epoch 11/15
Train Loss: 0.4348, Acc: 85.06%
Val Loss: 0.8074, Acc: 74.70%
Epoch 12/15
Train Loss: 0.4261, Acc: 85.28%
Val Loss: 0.7970, Acc: 75.18%
Epoch 13/15
Train Loss: 0.4113, Acc: 85.72%
Val Loss: 0.7827, Acc: 75.33%
Epoch 14/15
Train Loss: 0.4088, Acc: 85.66%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇██████████
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▆▇▇▇██████████
val_loss,█▃▂▂▁▁▂▁▁▁▁▁▁▁▁
epoch,14
train_acc,85.96733
train_loss,0.39715
val_acc,75.1888
val_loss,0.79417


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2ixqhn81 with config:
wandb: 	beam_size: 3
wandb: 	cell_type: GRU
wandb: 	dec_layers: 3
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 64
wandb: 	enc_layers: 3
wandb: 	epochs: 5
wandb: 	hidden_dim: 64


...

Epoch 1/15
Train Loss: 1.2460, Acc: 61.52%
Val Loss: 0.9307, Acc: 69.60%
Epoch 2/15
Train Loss: 0.7007, Acc: 77.43%
Val Loss: 0.8519, Acc: 72.06%
Epoch 3/15
Train Loss: 0.6100, Acc: 80.14%
Val Loss: 0.8310, Acc: 72.93%
Epoch 4/15
Train Loss: 0.5634, Acc: 81.56%
Val Loss: 0.8386, Acc: 72.93%
Epoch 5/15
Train Loss: 0.5345, Acc: 82.34%
Val Loss: 0.8077, Acc: 73.68%
Epoch 6/15
Train Loss: 0.5144, Acc: 82.96%
Val Loss: 0.8220, Acc: 73.77%
Epoch 7/15
Train Loss: 0.4985, Acc: 83.37%
Val Loss: 0.8059, Acc: 74.37%
Epoch 8/15
Train Loss: 0.4906, Acc: 83.47%
Val Loss: 0.8029, Acc: 74.74%
Epoch 9/15
Train Loss: 0.4769, Acc: 83.83%
Val Loss: 0.7883, Acc: 74.73%
Epoch 10/15
Train Loss: 0.4699, Acc: 84.05%
Val Loss: 0.8301, Acc: 74.33%
Epoch 11/15
Train Loss: 0.4638, Acc: 84.21%
Val Loss: 0.8094, Acc: 74.88%
Epoch 12/15
Train Loss: 0.4563, Acc: 84.35%
Val Loss: 0.7953, Acc: 74.24%
Epoch 13/15
Train Loss: 0.4531, Acc: 84.48%
Val Loss: 0.8083, Acc: 74.52%
Epoch 14/15
Train Loss: 0.4428, Acc: 84.74%
Val

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁
val_acc,▁▄▅▅▆▇▇██▇█▇█▇█
val_loss,█▄▃▃▂▃▂▂▁▃▂▁▂▂▄
epoch,14
train_acc,84.87337
train_loss,0.43811
val_acc,74.63904
val_loss,0.84283


wandb: Agent Starting Run: mv89x1co with config:
wandb: 	beam_size: 5
wandb: 	cell_type: GRU
wandb: 	dec_layers: 3
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 64
wandb: 	enc_layers: 3
wandb: 	epochs: 5
wandb: 	hidden_dim: 256


Epoch 1/5
Train Loss: 1.2265, Acc: 60.90%
Val Loss: 0.9564, Acc: 68.44%
Epoch 2/5
Train Loss: 0.7646, Acc: 75.01%
Val Loss: 0.8854, Acc: 71.33%
Epoch 3/5
Train Loss: 0.6826, Acc: 77.61%
Val Loss: 0.8508, Acc: 72.69%
Epoch 4/5
Train Loss: 0.6403, Acc: 79.04%
Val Loss: 0.8639, Acc: 72.75%
Epoch 5/5
Train Loss: 0.6128, Acc: 79.90%
Val Loss: 0.8203, Acc: 73.61%


epoch,▁▃▅▆█
train_acc,▁▆▇██
train_loss,█▃▂▁▁
val_acc,▁▅▇▇█
val_loss,█▄▃▃▁
epoch,4
train_acc,79.90023
train_loss,0.61277
val_acc,73.61477
val_loss,0.82032


In [4]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'embedding_dim': {'values': [64,128,256]},
        'hidden_dim': {'values': [64, 128,256 ]},
        'enc_layers': {'values': [1, 2,3]},
        'dec_layers': {'values': [1, 2,3]},
        'cell_type': {'values': ['GRU', 'LSTM', 'RNN']},
        'dropout': {'values': [0.2, 0.3,0.5]},
        'epochs': {'values': [20, 15]},
        'beam_size': {'values': [1, 3, 5]}
    }
}
# Sweep
# =======================
default_config = {
    'embedding_dim': 32,
    'hidden_dim': 64,
    'enc_layers': 1,
    'dec_layers': 1,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 10,
    'beam_size': 1
}


# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        inp_vocab.build([p[0] for p in self.pairs])
        out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder and Decoder
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

# =======================
# Seq2Seq Model with Beam Search
# =======================
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        output_dim = output.shape[-1]
        loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Train Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            output_dim = output.shape[-1]
            loss = criterion(output[:, 1:].reshape(-1, output_dim), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    acc = 100.0 * total_correct / total_count
    print(f"Val Loss: {total_loss / len(loader):.4f}, Acc: {acc:.2f}%")
    return total_loss / len(loader), acc

# =======================
# Main
# =======================
def main():
    wandb.init(config=default_config, project="dakshina-transliteration")
    config = wandb.config
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
    dev_path = "/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
    train_data = TransliterationDataset(train_path, inp_vocab, out_vocab)
    dev_data = TransliterationDataset(dev_path, inp_vocab, out_vocab)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
    decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
    model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)
        wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch+1})

# =======================

if __name__ == '__main__':
    sweep_id = wandb.sweep(sweep_config, project="dakshina-transliteration")
    wandb.agent(sweep_id, function=main,count=10)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: q6l1non5
Sweep URL: https://wandb.ai/manglesh_dl_ass3/dakshina-transliteration/sweeps/q6l1non5


wandb: Agent Starting Run: t64a62c3 with config:
wandb: 	beam_size: 5
wandb: 	cell_type: RNN
wandb: 	dec_layers: 2
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 128
wandb: 	enc_layers: 3
wandb: 	epochs: 20
wandb: 	hidden_dim: 256
wandb: Currently logged in as: manglesh_dlass3 (manglesh_dl_ass3) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1/20
Train Loss: 1.9554, Acc: 42.42%
Val Loss: 1.7184, Acc: 46.60%
Epoch 2/20
Train Loss: 1.5413, Acc: 52.60%
Val Loss: 1.5627, Acc: 50.22%
Epoch 3/20
Train Loss: 1.3921, Acc: 56.85%
Val Loss: 1.4492, Acc: 53.92%
Epoch 4/20
Train Loss: 1.3088, Acc: 59.20%
Val Loss: 1.3864, Acc: 55.18%
Epoch 5/20
Train Loss: 1.2589, Acc: 60.54%
Val Loss: 1.3533, Acc: 56.45%
Epoch 6/20
Train Loss: 1.2227, Acc: 61.63%
Val Loss: 1.3185, Acc: 57.52%
Epoch 7/20
Train Loss: 1.1909, Acc: 62.43%
Val Loss: 1.2978, Acc: 57.83%
Epoch 8/20
Train Loss: 1.1754, Acc: 62.80%
Val Loss: 1.3139, Acc: 57.95%
Epoch 9/20
Train Loss: 1.1584, Acc: 63.26%
Val Loss: 1.3047, Acc: 58.31%
Epoch 10/20
Train Loss: 1.1579, Acc: 63.28%
Val Loss: 1.2658, Acc: 58.91%
Epoch 11/20
Train Loss: 1.1431, Acc: 63.65%
Val Loss: 1.2666, Acc: 58.62%
Epoch 12/20
Train Loss: 1.1346, Acc: 63.93%
Val Loss: 1.2665, Acc: 59.07%
Epoch 13/20
Train Loss: 1.1267, Acc: 64.30%
Val Loss: 1.2796, Acc: 58.97%
Epoch 14/20
Train Loss: 1.1270, Acc: 64.23%
Val

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_acc,▁▄▆▆▇▇▇▇████████████
train_loss,█▅▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▅▆▆▇▇▇▇█▇██▇████▇█
val_loss,█▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,20
train_acc,64.44335
train_loss,1.11939
val_acc,58.94795
val_loss,1.27341


...

epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
train_acc,▁▆▇▇▇▇█████████
train_loss,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▅▅▆▆▇▇▇▇██▇▇██
val_loss,█▄▄▂▃▃▂▃▂▁▂▂▃▂▂
epoch,15
train_acc,81.38486
train_loss,0.56288
val_acc,74.55224
val_loss,0.80338


wandb: Agent Starting Run: 1zswwpkt with config:
wandb: 	beam_size: 5
wandb: 	cell_type: GRU
wandb: 	dec_layers: 1
wandb: 	dropout: 0.5
wandb: 	embedding_dim: 64
wandb: 	enc_layers: 2
wandb: 	epochs: 20
wandb: 	hidden_dim: 256


Epoch 1/20


wandb: Ctrl + C detected. Stopping sweep.


Train Loss: 1.2082, Acc: 62.37%
Val Loss: 1.0177, Acc: 66.39%
Epoch 2/20
Train Loss: 0.7004, Acc: 77.47%
Val Loss: 0.9014, Acc: 70.19%
Epoch 3/20
Train Loss: 0.6098, Acc: 80.10%
Val Loss: 0.8823, Acc: 71.00%
Epoch 4/20


In [7]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import random
import numpy as np

# =======================
# Fixed Best Configuration
# =======================
config = {
    'embedding_dim': 128,
    'hidden_dim': 256,
    'enc_layers': 3,
    'dec_layers': 3,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 15,
    'beam_size': 5
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wandb.init(project="dakshina-transliteration", config=config)
config = wandb.config

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab=None, out_vocab=None):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if inp_vocab and out_vocab:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens)

# =======================
# Encoder / Decoder / Seq2Seq
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)

        enc_hidden = self.encoder(src[0], src[1])

        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)

        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if random.random() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

# =======================
# Train / Evaluate
# =======================
def compute_accuracy(preds, targets):
    preds = preds.argmax(-1)
    correct = ((preds == targets) & (targets != 0)).sum().item()
    total = (targets != 0).sum().item()
    return correct / total

def train_eval(model, loader, criterion, optimizer, is_train):
    model.train() if is_train else model.eval()
    total_loss, total_acc = 0, 0
    with torch.set_grad_enabled(is_train):
        for src, trg, src_lens, _ in loader:
            src, trg = src.to(device), trg.to(device)
            if is_train: optimizer.zero_grad()
            output = model((src, src_lens), trg)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            acc = compute_accuracy(output[:, 1:], trg[:, 1:])
            if is_train:
                loss.backward()
                optimizer.step()
            total_loss += loss.item()
            total_acc += acc
    return total_loss / len(loader), total_acc / len(loader)

# =======================
# Train and Save Best
# =======================
inp_vocab, out_vocab = Vocab(), Vocab()
train_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
dev_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
test_set = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv", inp_vocab, out_vocab)

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_set, batch_size=32, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, collate_fn=collate_fn)

encoder = Encoder(inp_vocab.size, config.embedding_dim, config.hidden_dim, config.enc_layers, config.cell_type, config.dropout)
decoder = Decoder(out_vocab.size, config.embedding_dim, config.hidden_dim, config.dec_layers, config.cell_type, config.dropout)
model = Seq2Seq(encoder, decoder, config.enc_layers, config.dec_layers, config.cell_type, device).to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

best_val_acc = 0.0
for epoch in range(config.epochs):
    train_loss, train_acc = train_eval(model, train_loader, criterion, optimizer, is_train=True)
    val_loss, val_acc = train_eval(model, dev_loader, criterion, optimizer, is_train=False)
    wandb.log({"train_loss": train_loss, "train_acc": train_acc, "val_loss": val_loss, "val_acc": val_acc, "epoch": epoch})
    print(f"Epoch {epoch}: Train Loss={train_loss:.4f} Acc={train_acc:.4f}, Val Loss={val_loss:.4f} Acc={val_acc:.4f}")
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best_model.pt")

# =======================
# Test Evaluation
# =======================
model.load_state_dict(torch.load("best_model.pt"))
model.eval()

os.makedirs("predictions_vanilla", exist_ok=True)
with open("predictions_vanilla/preds.txt", "w", encoding="utf-8") as f:
    correct, total = 0, 0
    samples = []
    for src, trg, src_lens, _ in test_loader:
        src, trg = src.to(device), trg.to(device)
        output = model((src, src_lens), trg, teacher_forcing_ratio=0.0)
        pred_idxs = output.argmax(-1)[0].tolist()
        true_idxs = trg[0].tolist()
        pred_str = out_vocab.decode(pred_idxs)
        true_str = out_vocab.decode(true_idxs)
        input_str = inp_vocab.decode(src[0].tolist())
        f.write(f"{input_str}\t{true_str}\t{pred_str}\n")
        if pred_str == true_str:
            correct += 1
        total += 1
        samples.append((input_str, true_str, pred_str))
    test_acc = correct / total
    wandb.log({"test_accuracy": test_acc})
    print("Test Accuracy:", test_acc)

# Sample Grid (for visualization)
print("\nSample Predictions:")
print("{:<20} | {:<20} | {:<20}".format("Input", "Reference", "Prediction"))
print("=" * 65)
for s in random.sample(samples, 10):
    print("{:<20} | {:<20} | {:<20}".format(*s))


Epoch 0: Train Loss=1.2103 Acc=0.6221, Val Loss=0.7996 Acc=0.7455
Epoch 1: Train Loss=0.7029 Acc=0.7740, Val Loss=0.6784 Acc=0.7765
Epoch 2: Train Loss=0.6081 Acc=0.8035, Val Loss=0.6208 Acc=0.8025
Epoch 3: Train Loss=0.5603 Acc=0.8173, Val Loss=0.6208 Acc=0.8002
Epoch 4: Train Loss=0.5335 Acc=0.8238, Val Loss=0.6086 Acc=0.8038
Epoch 5: Train Loss=0.5105 Acc=0.8311, Val Loss=0.6006 Acc=0.8093
Epoch 6: Train Loss=0.4914 Acc=0.8355, Val Loss=0.5927 Acc=0.8080
Epoch 7: Train Loss=0.4737 Acc=0.8404, Val Loss=0.6052 Acc=0.8078
Epoch 8: Train Loss=0.4590 Acc=0.8435, Val Loss=0.6019 Acc=0.8034
Epoch 9: Train Loss=0.4495 Acc=0.8457, Val Loss=0.5979 Acc=0.8129
Epoch 10: Train Loss=0.4416 Acc=0.8482, Val Loss=0.6096 Acc=0.8070
Epoch 11: Train Loss=0.4337 Acc=0.8492, Val Loss=0.6045 Acc=0.8113
Epoch 12: Train Loss=0.4265 Acc=0.8510, Val Loss=0.5946 Acc=0.8153
Epoch 13: Train Loss=0.4222 Acc=0.8518, Val Loss=0.6018 Acc=0.8093
Epoch 14: Train Loss=0.4120 Acc=0.8546, Val Loss=0.6180 Acc=0.8116
Epoch

In [18]:
# =======================
# Imports and Sweep Config
# =======================
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import wandb
import os
import math
from tqdm import tqdm

os.makedirs("predictions_vanilla", exist_ok=True)

best_config = {
    'embedding_dim': 128,
    'hidden_dim': 256,
    'enc_layers': 3,
    'dec_layers': 3,
    'cell_type': 'LSTM',
    'dropout': 0.2,
    'epochs': 15,
    'beam_size': 5
}

# =======================
# Vocabulary
# =======================
class Vocab:
    def __init__(self):
        self.char2idx = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
        self.idx2char = {0: "<pad>", 1: "<sos>", 2: "<eos>"}
        self.size = 3

    def build(self, texts):
        for text in texts:
            for char in text:
                if char not in self.char2idx:
                    self.char2idx[char] = self.size
                    self.idx2char[self.size] = char
                    self.size += 1

    def encode(self, text):
        return [self.char2idx[c] for c in text]

    def decode(self, idxs):
        return ''.join([self.idx2char[i] for i in idxs if i > 2])

# =======================
# Dataset
# =======================
class TransliterationDataset(Dataset):
    def __init__(self, filepath, inp_vocab, out_vocab, is_test=False):
        self.pairs = []
        with open(filepath, encoding='utf-8') as f:
            for line in f:
                fields = line.strip().split('\t')
                if len(fields) < 2:
                    continue
                lat, dev = fields[0], fields[1]
                self.pairs.append((lat, dev))
        if not is_test:
            inp_vocab.build([p[0] for p in self.pairs])
            out_vocab.build([p[1] for p in self.pairs])
        self.inp_vocab = inp_vocab
        self.out_vocab = out_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        lat, dev = self.pairs[idx]
        x = self.inp_vocab.encode(lat)
        y = [self.out_vocab.char2idx["<sos>"]] + self.out_vocab.encode(dev) + [self.out_vocab.char2idx["<eos>"]]
        return torch.tensor(x), torch.tensor(y), lat, dev

def collate_fn(batch):
    x_batch, y_batch, lat, dev = zip(*batch)
    x_lens = [len(x) for x in x_batch]
    y_lens = [len(y) for y in y_batch]
    x_pad = nn.utils.rnn.pad_sequence(x_batch, batch_first=True, padding_value=0)
    y_pad = nn.utils.rnn.pad_sequence(y_batch, batch_first=True, padding_value=0)
    return x_pad, y_pad, torch.tensor(x_lens), torch.tensor(y_lens), lat, dev

# =======================
# Encoder, Decoder, Seq2Seq
# =======================
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)

    def forward(self, x, lengths):
        embedded = self.embedding(x)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        outputs, hidden = self.rnn(packed)
        return hidden

class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, num_layers, cell_type, dropout):
        super().__init__()
        self.cell_type = cell_type
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        rnn_class = {"GRU": nn.GRU, "LSTM": nn.LSTM, "RNN": nn.RNN}[cell_type]
        self.rnn = rnn_class(emb_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_token, hidden):
        embedded = self.embedding(input_token).unsqueeze(1)
        output, hidden = self.rnn(embedded, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, enc_layers, dec_layers, cell_type, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.size()
        vocab_size = self.decoder.out.out_features
        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)
        enc_hidden = self.encoder(src[0], src[1])
        if self.cell_type == "LSTM":
            h, c = enc_hidden
            h = self._match_layers(h)
            c = self._match_layers(c)
            dec_hidden = (h, c)
        else:
            dec_hidden = self._match_layers(enc_hidden)
        input_token = trg[:, 0]
        for t in range(1, trg_len):
            output, dec_hidden = self.decoder(input_token, dec_hidden)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1
        return outputs

    def _match_layers(self, hidden):
        if self.enc_layers == self.dec_layers:
            return hidden
        elif self.enc_layers > self.dec_layers:
            return hidden[:self.dec_layers]
        else:
            pad = hidden.new_zeros((self.dec_layers - self.enc_layers, *hidden.shape[1:]))
            return torch.cat([hidden, pad], dim=0)

    def predict(self, src_tensor, src_len, max_len=30):
        self.eval()
        with torch.no_grad():
            enc_hidden = self.encoder(src_tensor.unsqueeze(0), torch.tensor([src_len]))
            if self.cell_type == "LSTM":
                h, c = enc_hidden
                h = self._match_layers(h)
                c = self._match_layers(c)
                dec_hidden = (h, c)
            else:
                dec_hidden = self._match_layers(enc_hidden)
            input_token = torch.tensor([2]).to(self.device)  # <sos>
            output_seq = []
            for _ in range(max_len):
                output, dec_hidden = self.decoder(input_token, dec_hidden)
                top1 = output.argmax(1)
                if top1.item() == 2: break  # <eos>
                output_seq.append(top1.item())
                input_token = top1
        return output_seq

# =======================
# Train & Eval
# =======================
def train(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct, total_count = 0, 0, 0
    for src, trg, src_lens, _, _, _ in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model((src, src_lens), trg)
        loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
        pred = output.argmax(2)
        correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
        total_correct += correct
        total_count += (trg[:, 1:] != 0).sum().item()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct, total_count = 0, 0, 0
    with torch.no_grad():
        for src, trg, src_lens, _, _, _ in loader:
            src, trg = src.to(device), trg.to(device)
            output = model((src, src_lens), trg, teacher_forcing_ratio=0)
            loss = criterion(output[:, 1:].reshape(-1, output.shape[-1]), trg[:, 1:].reshape(-1))
            pred = output.argmax(2)
            correct = ((pred[:, 1:] == trg[:, 1:]) & (trg[:, 1:] != 0)).sum().item()
            total_correct += correct
            total_count += (trg[:, 1:] != 0).sum().item()
            total_loss += loss.item()
    return total_loss / len(loader), 100.0 * total_correct / total_count

# =======================
# Main
# =======================
# =======================
# Main
# =======================
def main():
    wandb.init(project="dakshina-transliteration")
    config = best_config  # Use best_config directly and avoid wandb.config

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    inp_vocab, out_vocab = Vocab(), Vocab()
    train_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv", inp_vocab, out_vocab)
    dev_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv", inp_vocab, out_vocab)
    test_data = TransliterationDataset("/kaggle/input/dataset-01/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv", inp_vocab, out_vocab, is_test=True)

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_data, batch_size=32, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate_fn)

    encoder = Encoder(inp_vocab.size, config['embedding_dim'], config['hidden_dim'],
                      config['enc_layers'], config['cell_type'], config['dropout'])
    decoder = Decoder(out_vocab.size, config['embedding_dim'], config['hidden_dim'],
                      config['dec_layers'], config['cell_type'], config['dropout'])
    model = Seq2Seq(encoder, decoder, config['enc_layers'], config['dec_layers'], config['cell_type'], device).to(device)

    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    best_val_acc = 0.0
    for epoch in range(config['epochs']):
        print(f"Epoch {epoch+1}/{config['epochs']}")
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, dev_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        })

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ Best model saved.")

    # Evaluation on test set
    print("\n🔍 Evaluating on test set with best model:")
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    total_correct, total_count = 0, 0

    with open("predictions_vanilla/test_predictions.txt", "w", encoding="utf-8") as f:
        for src, _, src_lens, _, lat, gold in test_loader:
            src = src.to(device)
            pred_ids = model.predict(src[0], src_lens[0].item())
            pred = out_vocab.decode(pred_ids)
            f.write(f"{lat[0]}\t{gold[0]}\t{pred}\n")
            if pred == gold[0]:
                total_correct += 1
            total_count += 1

    test_acc = 100.0 * total_correct / total_count
    print(f"📊 Test Accuracy: {test_acc:.2f}%")
    wandb.log({"test_acc": test_acc})

if __name__ == "__main__":
    main()




train_acc,▁▆▇███
train_loss,█▃▂▂▁▁
val_acc,▁▆▇▇██
val_loss,█▄▂▂▁▂
train_acc,83.12333
train_loss,0.50879
val_acc,72.13333
val_loss,0.86891


Epoch 1/15
Train Loss: 1.2916, Accuracy: 60.52%
Val   Loss: 0.9746, Accuracy: 68.25%
✅ Best model saved.
Epoch 2/15
Train Loss: 0.6719, Accuracy: 78.24%
Val   Loss: 0.8701, Accuracy: 71.92%
✅ Best model saved.
Epoch 3/15
Train Loss: 0.5746, Accuracy: 81.27%
Val   Loss: 0.8041, Accuracy: 73.33%
✅ Best model saved.
Epoch 4/15
Train Loss: 0.5236, Accuracy: 82.72%
Val   Loss: 0.8114, Accuracy: 73.86%
✅ Best model saved.
Epoch 5/15
Train Loss: 0.4914, Accuracy: 83.66%
Val   Loss: 0.8050, Accuracy: 74.60%
✅ Best model saved.
Epoch 6/15
Train Loss: 0.4701, Accuracy: 84.21%
Val   Loss: 0.8052, Accuracy: 74.57%
Epoch 7/15
Train Loss: 0.4553, Accuracy: 84.49%
Val   Loss: 0.8100, Accuracy: 74.55%
Epoch 8/15
Train Loss: 0.4394, Accuracy: 84.89%
Val   Loss: 0.7896, Accuracy: 74.93%
✅ Best model saved.
Epoch 9/15
Train Loss: 0.4285, Accuracy: 85.05%
Val   Loss: 0.7989, Accuracy: 74.77%
Epoch 10/15
Train Loss: 0.4173, Accuracy: 85.34%
Val   Loss: 0.7853, Accuracy: 75.38%
✅ Best model saved.
Epoch 11/